In [0]:
from datetime import datetime
dbutils.widgets.text("sales_table_name", "bronze_sales", "Source Sales Table")
dbutils.widgets.text("archive_path_base", "dbfs:/FileStore/lab_data/archived_sales/", "Base Archive Path")
dbutils.widgets.text("database_name", "module2_db", "Database Name")
dbutils.widgets.text("validator_task_key", "Task_Validate_Sales", "Task key of the validation notebook")

sales_table = dbutils.widgets.get("sales_table_name")
archive_path_base = dbutils.widgets.get("archive_path_base")
db_name = dbutils.widgets.get("database_name")
validator_task_key = dbutils.widgets.get("validator_task_key")
full_sales_table_name = f"{db_name}.{sales_table}"

timestamp_str = datetime.now().strftime("%Y%m%d%H%M%S")
archive_path_final = f"{archive_path_base.rstrip('/')}/{timestamp_str}/"

print(f"Archiving table {full_sales_table_name} to {archive_path_final}")

try:
    # Get values for context from the validation task
    validation_status = dbutils.jobs.taskValues.get(taskKey=validator_task_key, key="validation_status", default="N/A_STATUS", debugValue="DEBUG_ARCHIVE")
    source_count = dbutils.jobs.taskValues.get(taskKey=validator_task_key, key="source_table_record_count", default=0, debugValue=0)
    print(f"Upstream validation status for context: {validation_status}")
    print(f"Upstream source record count for context: {source_count}")
except Exception as e:
    print(f"Could not retrieve some task values from '{validator_task_key}': {e}.")

try:
    df_sales = spark.table(full_sales_table_name)
    df_sales.write.format("parquet").mode("overwrite").save(archive_path_final)
    archived_count = df_sales.count()
    print(f"Successfully archived {archived_count} records from {full_sales_table_name} to {archive_path_final}")
    dbutils.jobs.taskValues.set(key="archived_path", value=archive_path_final)
    dbutils.jobs.taskValues.set(key="archived_count", value=archived_count)
    dbutils.notebook.exit(f"Archiving task finished. Path: {archive_path_final}")
except Exception as e:
    error_msg = f"Failed to archive {full_sales_table_name}: {e}"
    print(error_msg)
    dbutils.notebook.exit(error_msg)